In [24]:
import tensorflow as tf
print(tf.__version__)

2.5.0


## 0. Imports

In [25]:
import pandas as pd
import numpy as np
import os
import logging
from google.cloud.storage import Client
from sklearn.model_selection import train_test_split
import datetime
import gcsfs

## 1. Config work

In [26]:
nowTime  = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [27]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT=PROJECT_ID[0]
BUCKET_NAME = f"{PROJECT}-tweet-sentiment-analysis-test"
BUCKET= f"gs://{PROJECT}-tweet-sentiment-analysis-test"
ROOT='sentiment-classifcation'
MODEL_DIR=os.path.join(ROOT,'models_level0').replace("\\","/")
PACKAGES_DIR=os.path.join(ROOT,'packages_level0').replace("\\","/")
REGION = 'europe-west1'

if not os.path.exists('./model-'+nowTime+'/'):
    os.makedirs('./model-'+nowTime+'/')
temp_model = './model-'+nowTime+'/'

In [28]:
!gcloud config set project {PROJECT}

Updated property [core/project].


In [29]:
!gsutil mb -l {REGION} {BUCKET}

Creating gs://rare-result-248415-tweet-sentiment-analysis-test/...
ServiceException: 409 A Cloud Storage bucket named 'rare-result-248415-tweet-sentiment-analysis-test' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [30]:
!gsutil rm -r {BUCKET}/{ROOT}

Removing gs://rare-result-248415-tweet-sentiment-analysis-test/sentiment-classifcation/models_level0/model-2022-05-23-08-45-23/processor_state.pkl#1653297684002439...
Removing gs://rare-result-248415-tweet-sentiment-analysis-test/sentiment-classifcation/models_level0/model-2022-05-23-08-45-23/saved_model.pb#1653297684116628...
Removing gs://rare-result-248415-tweet-sentiment-analysis-test/sentiment-classifcation/models_level0/model-2022-05-23-08-45-23/variables/variables.data-00000-of-00001#1653297684540101...
Removing gs://rare-result-248415-tweet-sentiment-analysis-test/sentiment-classifcation/models_level0/model-2022-05-23-08-45-23/variables/variables.index#1653297684185974...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m 

In [31]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

## 2. Get data

### 2.1. Input data

In [32]:
sentiment_mapping={
    0:"negative",
    2:"neutral",
    4:"positive"
}

data can be downloaded from: https://www.kaggle.com/kazanova/sentiment140

In [33]:
df_twitter = pd.read_csv(BUCKET+"/Data/sentiment_140/training.csv",encoding="latin1", header=None)\
             .rename(columns={
                 0:"sentiment",
                 1:"id",
                 2:"time",
                 3:"query",
                 4:"username",
                 5:"text"
             })[["sentiment","text"]]

In [34]:
df_twitter["sentiment_label"] = df_twitter["sentiment"].map(sentiment_mapping)

### 2.2. Data processing fn

In [35]:
%%writefile preprocess.py

from tensorflow.python.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text
import re

class TextPreprocessor(object):
    def _clean_line(self, text):
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"@[A-Za-z0-9]+", "", text)
        text = re.sub(r"#[A-Za-z0-9]+", "", text)
        text = text.replace("RT","")
        text = text.lower()
        text = text.strip()
        return text
    
    def __init__(self, vocab_size, max_sequence_length):
        self._vocab_size = vocab_size
        self._max_sequence_length = max_sequence_length
        self._tokenizer = None

    def fit(self, text_list):        
        # Create vocabulary from input corpus.
        text_list_cleaned = [self._clean_line(txt) for txt in text_list]
        tokenizer = text.Tokenizer(num_words=self._vocab_size)
        tokenizer.fit_on_texts(text_list)
        self._tokenizer = tokenizer

    def transform(self, text_list):        
        # Transform text to sequence of integers
        text_list = [self._clean_line(txt) for txt in text_list]
        text_sequence = self._tokenizer.texts_to_sequences(text_list)

        # Fix sequence length to max value. Sequences shorter than the length are
        # padded in the beginning and sequences longer are truncated
        # at the beginning.
        padded_text_sequence = sequence.pad_sequences(
          text_sequence, maxlen=self._max_sequence_length)
        return padded_text_sequence

Overwriting preprocess.py


Some small test:

In [36]:
from preprocess import TextPreprocessor

processor = TextPreprocessor(5, 5)
processor.fit(['hello machine learning','test'])
processor.transform(['hello machine learning',"lol"])

array([[0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0]], dtype=int32)

### 2.3. Prep data

In [37]:
CLASSES = {'negative':0, 'positive': 1}  # label-to-int mapping
VOCAB_SIZE = 25000  # Limit on the number vocabulary size used for tokenization
MAX_SEQUENCE_LENGTH = 50  # Sentences will be truncated/padded to this length

In [38]:
sents = df_twitter.text
labels = np.array(df_twitter.sentiment_label.map(CLASSES))

# Train and test split
X, X_test, y, y_test = train_test_split(sents, labels, test_size=0.2)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.1)

# Create vocabulary from training corpus.
processor = TextPreprocessor(VOCAB_SIZE, MAX_SEQUENCE_LENGTH)
processor.fit(X_train)

# Preprocess the data
train_texts_vectorized = processor.transform(X_train)
eval_texts_vectorized = processor.transform(X_test)
validation_texts_vectorized = processor.transform(X_validation)

In [39]:
import pickle

with open('./model-'+nowTime+'/processor_state.pkl', 'wb') as f:
    pickle.dump(processor, f)

## 3. Model

In [40]:
# model parameters
LEARNING_RATE=.001
EMBEDDING_DIM=50
FILTERS=64
DROPOUT_RATE=0.5
POOL_SIZE=3
NUM_EPOCH=5
BATCH_SIZE=128
KERNEL_SIZES=[2,5,8]

### 3.1. Basic model

In [41]:
def create_model(vocab_size, embedding_dim, filters, kernel_sizes, dropout_rate, pool_size, embedding_matrix):
    
    # Input layer
    model_input = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

    # Embedding layer
    z = tf.keras.layers.Embedding(
        input_dim=vocab_size+1,
        output_dim=embedding_dim,
        input_length=MAX_SEQUENCE_LENGTH,
        weights=[embedding_matrix]
    )(model_input)

    z = tf.keras.layers.Dropout(dropout_rate)(z)

    # Convolutional block
    conv_blocks = []
    for kernel_size in kernel_sizes:
        conv = tf.keras.layers.Convolution1D(
            filters=filters,
            kernel_size=kernel_size,
            padding="valid",
            activation="relu",
            bias_initializer='random_uniform',
            strides=1)(z)
        conv = tf.keras.layers.MaxPooling1D(pool_size=2)(conv)
        conv = tf.keras.layers.Flatten()(conv)
        conv_blocks.append(conv)
        
    z = tf.keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

    z = tf.keras.layers.Dropout(dropout_rate)(z)
    z = tf.keras.layers.Dense(100, activation="relu")(z)
    model_output = tf.keras.layers.Dense(1, activation="sigmoid")(z)

    model = tf.keras.models.Model(model_input, model_output)
    
    return model

### 3.2. Pretrained Glove embeddings

embedding can be downloaded here: https://nlp.stanford.edu/projects/glove/

In [42]:
client = Client()
bucket = client.get_bucket(BUCKET_NAME)
temp_folder = "Data/embeddings/"
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)
blob = bucket.get_blob("Data/embeddings/glove.twitter.27B.50d.txt")
downloaded_file = blob.download_to_filename(temp_folder+'/glove.twitter.27B.50d.txt')

In [43]:
def get_coaefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coaefs(*o.strip().split()) for o in
                                                open(temp_folder+"/glove.twitter.27B.50d.txt","r",encoding="utf8"))

In [44]:
word_index = processor._tokenizer.word_index
nb_words = min(VOCAB_SIZE, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= VOCAB_SIZE: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

### 3.3. Create - compile - train

In [45]:
model = create_model(VOCAB_SIZE, EMBEDDING_DIM, FILTERS, KERNEL_SIZES, DROPOUT_RATE,POOL_SIZE, embedding_matrix)

In [46]:
# Compile model with learning parameters.
optimizer = tf.keras.optimizers.Nadam(lr=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [51]:
#keras train
history = model.fit(
    train_texts_vectorized, 
    y_train, 
    epochs=NUM_EPOCH, 
    batch_size=BATCH_SIZE,
    validation_data=(validation_texts_vectorized, y_validation),
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_acc',
            min_delta=0.005,
            patience=3,
            factor=0.5),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.005, 
            patience=5, 
            verbose=0, 
            mode='auto'
        ),
        tf.keras.callbacks.History()
    ]
)

Epoch 1/5
9000/9000 - 376s - loss: 0.4129 - acc: 0.8114 - val_loss: 0.3984 - val_acc: 0.8184
Epoch 2/5
9000/9000 - 367s - loss: 0.4073 - acc: 0.8149 - val_loss: 0.3977 - val_acc: 0.8192
Epoch 4/5
9000/9000 - 370s - loss: 0.4049 - acc: 0.8155 - val_loss: 0.3950 - val_acc: 0.8209
Epoch 5/5
9000/9000 - 367s - loss: 0.4009 - acc: 0.8178 - val_loss: 0.3933 - val_acc: 0.8213


In [53]:
# test model : acc loss
[loss, acc] = model.evaluate(eval_texts_vectorized, y_test )

10000/10000 [==============================] - 37s 4ms/step - loss: 0.3930 - acc: 0.8221


In [55]:
scores = model.predict(eval_texts_vectorized)
predictions = np.array([int(np.round(i)) for i in scores ])
confusion_matrix=tf.math.confusion_matrix(predictions, y_test)


In [57]:
# confusion matrix
print (str("matrix-co : "+str(confusion_matrix)))

matrix-co : tf.Tensor(
[[134217  31122]
 [ 25800 128861]], shape=(2, 2), dtype=int32)


In [58]:
# std prediction 
np.std(scores)

0.34319982

In [59]:
with open("history.pkl",'wb') as file:
    pickle.dump(history.history,file)

In [61]:

tf.saved_model.save(model,temp_model)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: ./model-2022-05-23-09-48-41/assets


In [62]:
#copy file to gcp storage

In [63]:
!gsutil cp -r {temp_model} {BUCKET}/{MODEL_DIR}/

Copying file://./model-2022-05-23-09-48-41/processor_state.pkl [Content-Type=application/octet-stream]...
Copying file://./model-2022-05-23-09-48-41/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./model-2022-05-23-09-48-41/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./model-2022-05-23-09-48-41/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
- [4 files][ 48.6 MiB/ 48.6 MiB]                                                
Operation completed over 4 objects/48.6 MiB.                                     


## 4. Deployment

### 4.1. Prepare custom model prediction

In [64]:
%%writefile model_prediction.py
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.python.lib.io import file_io

class CustomModelPrediction(object):

    def __init__(self, model, processor):
        # Class gets instantiated with a trained model file and a persisted processor
        self._model = model
        self._processor = processor

    def _postprocess(self, predictions):
    # Create an output signature
        labels = ['negative', 'positive']
        return [
            {
            "label":labels[int(np.round(prediction))],
            "score":float(np.round(prediction,4))
            } for prediction in predictions]

    def predict(self, instances, **kwargs):
    # Clean the data, make predictions and postprocess
        preprocessed_data = self._processor.transform(instances)
        predictions =  self._model.predict(preprocessed_data)
        labels = self._postprocess(predictions)
        return labels

    @classmethod
    def from_path(cls, model_dir):
    # Load the keras model and the persisted processor
        
        print ('test model')
        model = tf.keras.models.load_model(model_dir,custom_objects={'tf': tf})
    
    # I know, pickle is bad and I should feel bad
    
        with file_io.FileIO(os.path.join(model_dir, 'processor_state.pkl'), 'rb') as f:
            processor = pickle.load(f)

        return cls(model, processor)

Overwriting model_prediction.py


Test

In [65]:
requests = (["God I hate the north","god I love this"])

In [67]:
from model_prediction import CustomModelPrediction

classifier = CustomModelPrediction.from_path( BUCKET+'/'+MODEL_DIR+temp_model[1:])
results = classifier.predict(requests)
results

test model


[{'label': 'negative', 'score': 0.03550000116229057},
 {'label': 'positive', 'score': 0.9226999878883362}]

### 4.2. Package it

In [68]:
%%writefile setup.py

from setuptools import setup

REQUIRED_PACKAGES = ['gcsfs']
setup(
    name="tweet_sentiment_classifier",
    packages=[],
    include_package_data=False,
    version="1",
    scripts=["preprocess.py", "model_prediction.py"]
)

Overwriting setup.py


Wrap it up and copy to GCP

In [69]:
!python setup.py sdist --formats=gztar
!gsutil cp ./dist/tweet_sentiment_classifier-1.tar.gz {BUCKET}/{PACKAGES_DIR}/tweet_sentiment_classifier-1.tar.gz

running sdist
running egg_info
creating tweet_sentiment_classifier.egg-info
writing tweet_sentiment_classifier.egg-info/PKG-INFO
writing dependency_links to tweet_sentiment_classifier.egg-info/dependency_links.txt
writing top-level names to tweet_sentiment_classifier.egg-info/top_level.txt
writing manifest file 'tweet_sentiment_classifier.egg-info/SOURCES.txt'
reading manifest file 'tweet_sentiment_classifier.egg-info/SOURCES.txt'
writing manifest file 'tweet_sentiment_classifier.egg-info/SOURCES.txt'

running check


creating tweet_sentiment_classifier-1
creating tweet_sentiment_classifier-1/tweet_sentiment_classifier.egg-info
copying files to tweet_sentiment_classifier-1...
copying model_prediction.py -> tweet_sentiment_classifier-1
copying preprocess.py -> tweet_sentiment_classifier-1
copying setup.py -> tweet_sentiment_classifier-1
copying tweet_sentiment_classifier.egg-info/PKG-INFO -> tweet_sentiment_classifier-1/tweet_sentiment_classifier.egg-info
copying tweet_sentiment_classif

## 5. Create model and version

In [70]:
MODEL_NAME='tweet_sentiment_classifier_1'
VERSION_NAME='V1'
RUNTIME_VERSION='2.5' # tensorflow version
REGION='europe-west1'
id_model = 'model-'+nowTime

In [71]:
!gcloud ai-platform models create {MODEL_NAME} --regions {REGION}

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [rare-result-248415] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


In [72]:
!gcloud ai-platform models list --region global

Using endpoint [https://ml.googleapis.com/]
NAME                          DEFAULT_VERSION_NAME
tweet_sentiment_classifier
tweet_sentiment_classifier_1  V1


In [73]:
!gcloud ai-platform versions delete {VERSION_NAME} --model {MODEL_NAME} --region global --quiet

Using endpoint [https://ml.googleapis.com/]
Deleting version [V1]......done.                                               


In [75]:
!gcloud beta ai-platform versions create {VERSION_NAME} \
--model {MODEL_NAME} \
--origin {BUCKET}/{MODEL_DIR}/{id_model} \
--python-version 3.7 \
--runtime-version {RUNTIME_VERSION} \
--package-uris {BUCKET}/{PACKAGES_DIR}/tweet_sentiment_classifier-1.tar.gz \
--prediction-class=model_prediction.CustomModelPrediction \
--region global 

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [ ]:
## 6. Testing

In [76]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

In [77]:
requests = [
    "god this episode sucks",
    "meh, I kinda like it",
    "what were the writer thinking, omg!",
    "omg! what a twist, who would'v though :o!"
    "woohoow, sansa for the win!"
]

In [78]:
# JSON format the requests
request_data = {'instances': requests}

# Authenticate and call CMLE prediction API 
credentials = GoogleCredentials.get_application_default()

In [79]:
%%time

api = discovery.build('ml', 'v1')

parent = 'projects/{}/models/{}/versions/{}'.format(PROJECT, MODEL_NAME, VERSION_NAME)
parent = 'projects/{}/models/{}'.format(PROJECT, MODEL_NAME)
response = api.projects().predict(body=request_data, name=parent).execute()

CPU times: user 18.1 ms, sys: 4.04 ms, total: 22.1 ms
Wall time: 1.12 s


In [80]:
response["predictions"]

[{'label': 'negative', 'score': 0.008899999782443047},
 {'label': 'positive', 'score': 0.6571000218391418},
 {'label': 'positive', 'score': 0.6266000270843506},
 {'label': 'positive', 'score': 0.7268999814987183}]